In [1]:
import numpy as np
from scipy.stats import norm

# option parameters
S0 = 201.29  # stock price
K = 201.29  # strike price
vol = 0.6635  # volatility
r = 0.04924  # risk-free rate
T = 0.5  # time to maturity
div = 0.0  # dividend yield

# simulation parameters
N = 20  # number of time steps
M = 1000  # number of simulations

# time step size
dt = T / N

# generate stock price paths using Monte Carlo simulation
np.random.seed(0)
Z = norm.rvs(size=(M, N))
S = np.zeros((M, N+1))
S[:,0] = S0
for i in range(1, N+1):
    S[:,i] = S[:,i-1] * np.exp((r - div - 0.5*vol**2)*dt + vol*np.sqrt(dt)*Z[:,i-1])

# determine the exercise value at each time step
C = np.maximum(S - K, 0)  # call option
P = np.maximum(K - S, 0)  # put option

# use regression analysis to find the continuation value at each time step
for i in range(N-1, 0, -1):
    X = np.column_stack([np.ones(M), S[:,i], S[:,i]**2])
    Yc = np.exp(-r*dt) * C[:,i+1]
    Yp = np.exp(-r*dt) * P[:,i+1]
    coef_c = np.linalg.lstsq(X, Yc, rcond=None)[0]
    coef_p = np.linalg.lstsq(X, Yp, rcond=None)[0]
    C[:,i] = np.maximum(C[:,i], np.dot(X, coef_c))
    P[:,i] = np.maximum(P[:,i], np.dot(X, coef_p))

# determine the optimal exercise policy for each path
C_ex = np.zeros((M, N+1), dtype=bool)
P_ex = np.zeros((M, N+1), dtype=bool)
C_ex[:,N] = True
P_ex[:,N] = True
for i in range(N-1, 0, -1):
    C_ex[:,i] = np.logical_and(C[:,i] >= (C[:,i+1]-K), C_ex[:,i+1])
    P_ex[:,i] = np.logical_and(P[:,i] >= (P[:,i+1]-K), P_ex[:,i+1])

# calculate the expected payoff for each path
C_payoff = np.where(C_ex, C, 0)
P_payoff = np.where(P_ex, P, 0)
C_exp_payoff = np.mean(np.exp(-r*dt*np.arange(N+1))*C_payoff, axis=1)
P_exp_payoff = np.mean(np.exp(-r*dt*np.arange(N+1))*P_payoff, axis=1)

# discount the expected payoff to the present value to obtain the option prices
C_price = np.mean(C_exp_payoff)
P_price = np.mean(P_exp_payoff)

# print the results
print(f"American call option price: {C_price:.4f}")
print(f"American put option price: {P_price:.4f}")


American call option price: 45.2051
American put option price: 41.6566
